# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
cities=pd.read_csv("/Users/applewang/Desktop/github/api_challenge/cities.csv")
cityweather = pd.read_csv("/Users/applewang/Desktop/github/api_challenge/cityweather.csv")
cityweather

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#create location dataframe contain only lat and lng 
location_df = cityweather[["Latitude","Longitude"]]
location_df

,Latitude,Longitude
0,-45.87,170.50
1,71.69,128.87
2,45.10,-87.63
3,-6.22,-57.75
4,-23.12,-134.97
...,...,...
540,9.66,-68.58
541,24.09,32.91
542,54.10,123.57
543,4.41,114.01


In [4]:
humidity = cityweather["Humidity"]
humidity

0      93
1      57
2      72
3      49
4      53
       ..
540    59
541    10
542    66
543    94
544    39
Name: Humidity, Length: 545, dtype: int64

In [5]:
#config g-key
gmaps.configure(api_key = g_key)
#create layout list
layout = {'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'}
#create gmaps figure
fig = gmaps.figure(map_type="TERRAIN", zoom_level=1.5, center=(0,0), layout=layout)

humidity_layer = gmaps.heatmap_layer(location_df, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius= 1.5)

#jupyter nbextension enable --py gmaps

#heatmap
fig.add_layer(humidity_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
filtered_data = cityweather.loc[cityweather["Humidity"]<=50]
filtered_data = filtered_data.loc[filtered_data["Temperature"]>=65]
filtered_data = filtered_data.loc[filtered_data["Temperature"]<=85]
filtered_data = filtered_data.loc[filtered_data["Windspeed"]<=10]
print(filtered_data.count())


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = filtered_data
hotel_df['Hotel Name'] = ""
hotel_df['Hotel Lat'] = ""
hotel_df['Hotel Lng'] = ""
hotel_df['Hotel Rating'] = ""

In [ ]:
hotel_df.head()

In [ ]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
    }

In [14]:
for index, row in hotel_df.iterrows():
    
    
    lat = row["Latitude"]
    
    lng = row["Longitude"]
   
    city = row["City"]
 
    params["location"] = f'{lat},{lng}'
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
 
    hotel_api = requests.get(base_url, params=params)
    #making api request info in json format
    hotel_api = hotel_api.json()
    #exception to let know if hotel information for city list is not found, if found, hotel info added to dataframe(name, exact coords, and rating)
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_api["results"][0]["name"]
        hotel_df.loc[index, "Hotel Lat"] = hotel_api["results"][0]["geometry"]["location"]['lat']
        hotel_df.loc[index, "Hotel Lng"] = hotel_api["results"][0]["geometry"]["location"]['lng']
        hotel_df.loc[index, "Hotel Rating"] = hotel_api["results"][0]["rating"]
    #show what cities no information is found 
    except:
        print(f"Missing results for {city}, skipping entry")




Missing results for Albany, skipping entry
Missing results for Lodja, skipping entry
Missing results for Águia Branca, skipping entry
Missing results for Batagay, skipping entry
Missing results for Ittiri, skipping entry
Missing results for Belaya Gora, skipping entry
Missing results for Ushtobe, skipping entry


City            56
Country         54
Temperature     56
Humidity        56
Cloudiness      56
Windspeed       56
Date            56
Longitude       56
Latitude        56
Hotel Name      56
Hotel Lat       56
Hotel Lng       56
Hotel Rating    56
dtype: int64

In [17]:
hotel_df

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Date,Longitude,Latitude,Hotel Name,Hotel Lat,Hotel Lng,Hotel Rating
13,Beian,NO,73.40,46,71,5.82,1593022671,9.57,63.66,Orland Kysthotell,63.6859,9.66702,4.2
14,Port Alfred,ZA,68.00,33,0,5.01,1593022623,26.89,-33.59,The Halyards Hotel,-33.5943,26.8932,3.8
15,Albany,US,82.99,36,36,8.61,1593022627,-73.97,42.60,,,,
17,Mayo,US,84.00,42,1,6.93,1593022469,-76.51,38.89,Pirates Cove Restaurant and Dock Bar,38.8432,-76.5388,4.4
27,Lodja,CD,79.72,43,0,1.61,1593022642,23.43,-3.48,,,,
33,Nikolskoye,RU,73.99,49,31,4.47,1593022609,30.79,59.70,Tourist House - Sablino,59.6603,30.7785,4.2
35,Thompson,CA,80.60,32,40,4.70,1593022567,-97.86,55.74,Best Western Thompson Hotel & Suites,55.7457,-97.8516,4.3
66,Geseke,DE,78.01,47,0,6.93,1593022673,8.52,51.65,Rittergut Störmede KG,51.6375,8.46711,4.5
69,East London,ZA,70.95,29,0,7.92,1593022638,27.91,-33.02,Premier Hotels & Resorts,-33.0187,27.9187,4.2
71,São João da Barra,BR,80.60,44,84,9.17,1593022622,-41.05,-21.64,Pousada Kactus,-21.6654,-41.0232,4.6


In [36]:
NA_value = float("NaN")
hotel_df.replace("", NA_value, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head()

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Date,Longitude,Latitude,Hotel Name,Hotel Lat,Hotel Lng,Hotel Rating
13,Beian,NO,73.40,46,71,5.82,1593022671,9.57,63.66,Orland Kysthotell,63.685853,9.667020,4.2
14,Port Alfred,ZA,68.00,33,0,5.01,1593022623,26.89,-33.59,The Halyards Hotel,-33.594329,26.893248,3.8
17,Mayo,US,84.00,42,1,6.93,1593022469,-76.51,38.89,Pirates Cove Restaurant and Dock Bar,38.843185,-76.538843,4.4
33,Nikolskoye,RU,73.99,49,31,4.47,1593022609,30.79,59.70,Tourist House - Sablino,59.660258,30.778520,4.2
35,Thompson,CA,80.60,32,40,4.70,1593022567,-97.86,55.74,Best Western Thompson Hotel & Suites,55.745701,-97.851640,4.3


In [37]:
# NOTE: Do not change any of the code in this cell
#added a line to the box template to add the hotel rating to the info box!!!
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Rating</dt><dd>{Hotel Rating}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

In [38]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))